In [1]:
# Copyright 2022 NVIDIA Corporation. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions anda
# limitations under the License.
# ==============================================================================

# Each user is responsible for checking the content of datasets and the
# applicable licenses and determining if suitable for the intended use.

<img src="https://developer.download.nvidia.com/notebooks/dlsw-notebooks/merlin_models_entertainment-with-pretrained-embeddings/nvidia_logo.png" style="width: 90px; float: right;">

# Training with pretrained embeddings

## Overview

In this use case we will consider how we might train with pretrained embeddings.

Pretrained embeddings can allow our model to include information from additional modalities (for instance, we might want to grab CNN descriptors of product images). They can also come from other models that we train on our data. For example, we might train a word2vec model on the sequence of purchased items by a customer and want to include this information in our retrieval or ranking model.

The use cases are many, but this particular example will focus on the technical aspects of working with pretrained embeddings.

We will use a synthetic version of the MovieLens 100k dataset and emulate a scenario where we would have a pretrained embedding for each of the movies in the dataset.

### Learning objectives

- Training with pretrained embeddings
- Understanding [the Schema file](https://github.com/NVIDIA-Merlin/core/blob/stable/merlin/schema/schema.py)

## Downloading and preparing the dataset

In [2]:
import merlin.models.tf as mm
from merlin.schema.tags import Tags
import tensorflow as tf
from merlin.models.tf.blocks import *
from merlin.datasets.synthetic import generate_data
from merlin.dataloader.ops.embeddings import EmbeddingOperator

import numpy as np

In [3]:
train = generate_data('movielens-100k', num_rows=100_000)
train.schema = train.schema.excluding_by_name(["title"])

In [4]:
target_column = train.schema.select_by_name(["rating_binary"]).column_names[0]
target_column

# Passing the embeddings directly to our model using `TensorInitializer`

One way of passing the embeddings directly to our model is using the `TensorInitializer` as part of the `mm.Embeddings`.

This is a straightforward method that works well with small embedding tables.

Let's begin by looking at the schema which holds vital information about our dataset. We can extract the embedding table size for the `moveId` column from it.

In [5]:
train.schema['movieId'].properties['embedding_sizes']['cardinality']

From the schema, we can tell that the cardinality of `movieId` is 1680. Index 0 will be used in case an unknown `movieId` is encountered

In order to accommodate this, we initialize our embedding table of dimensionality of (1681, 128).

In [6]:
pretrained_movie_embs = np.random.random((1681, 128))

In [7]:
schema = train.schema

This is only a mock up embedding table. In reality, this is where we would pass our embeddings from another model.

The dimensionality of each embedding, that of 128, is arbitrary. We could have specified some other value here, though generally multiples of 8 tend to work well.

We need to update the schema properties of our `movieId` column since we will not be using the default embedding dimension.

In [8]:
schema['movieId'].properties['embedding_sizes'] = {
    'cardinality': float(pretrained_movie_embs.shape[0]), 
    'dimension': float(pretrained_movie_embs.shape[1])
}

In [9]:
train.schema = schema

In [10]:
train.schema

## Building the model

We now have everything we need to construct a model and train on our custom embeddings. In order to do so, we will leverage the `TensorInitializer` class and `Embeddings` function to set the `trainable` arg to `False`, so that our pre-trained embedddings will be frozen and not be updated during model training.

In [11]:
embed_dims = {}
embed_dims["movieId"] = pretrained_movie_embs.shape[1]

embeddings_init={
    "movieId": mm.TensorInitializer(pretrained_movie_embs),
}

embeddings_block = mm.Embeddings(
    train.schema.select_by_tag(Tags.CATEGORICAL),
    infer_embedding_sizes=True,
    embeddings_initializer=embeddings_init,
    trainable={'movieId': False},
    dim=embed_dims,
)
input_block = mm.InputBlockV2(train.schema, categorical=embeddings_block)

Let us now feed our input_block into our model.

In [12]:
model = mm.DCNModel(
    train.schema,
    depth=2,
    input_block=input_block,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryOutput(target_column)
)

We could have created the model without passing the `embeddings_block` to the `input_block`. The model would still be able to infer how to construct itself (what should be the dimensionality of the input layer and so on) from the information contained in the schema.

However, passing a `TensorInitializer` into the constructor of the `input_block` tells our model to use our embedding table (`pretrained_movie_embs`) for that particular column of our dataset (`movieId`) as opposed to the model randomly initializing a brand new embedding matrix. For categorical columns we do not provide this information, the model will go with the standard initialization logic, which is to create an embedding table of appropriate size and perform random preinitialization.

Additionally, we set the `trainable` parameter for our pre-trained embeddings to `False` to ensure the embeddings will not be modified during training.

## Training

We train our model with `AUC` as our metric.

As we use synthetic data, the AUC score will not improve significantly.

In [13]:
%%time
opt = tf.keras.optimizers.legacy.Adagrad(learning_rate=1e-1)
model.compile(optimizer=opt, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])

In [14]:
model.fit(train, batch_size=1024, epochs=5)

In [15]:
history = model.history.history

# Passing the `EmbeddingOperator` to the `Loader`

Another way of training with pretrained embeddings is to create a custom `Loader` and equip it with the ability to feed embeddings to our model.

When we use `mm.Embeddings` and `TensorInitializer` as above, the embeddings are moved to the GPU and can be considered part of our model. That might become problematic if the embedding table is large.

Taking the approach below the pretrained embeddings are passed to the model as part of each batch. We do not hold the embedding table in GPU memory, which depending on the scenario might consists of millions of rows.

We can reuse the train data and the embedding information we generated above.

Just as a reminder, these are the dimensions of our embeddings array.

In [16]:
pretrained_movie_embs.shape

We instantiate the `Loader` passing in the `EmbeddingOperator`. It encapsulates the recipe how to link the pretrained embeddings that we pass in (`pretrained_movie_embs`) to the data we provide as `train` (the association of records will be performed on the `movieId` column -- when a row with `movieId` of 10 is encountered, `pretrained_movie_embs[10]` will be feed along with that row).

In [17]:
loader = mm.Loader(
    train,
    batch_size=1024,
    transforms=[
        EmbeddingOperator(
            pretrained_movie_embs,
            lookup_key="movieId",
            embedding_name="pretrained_movie_embeddings",
        ),
    ],
)

Let us take a look at an example of a batch.

In [18]:
loader.peek()

We can seethe `pretrained_movie_embeddings` included in the batch and being represented as a `float64` `tf.Tensor`.

Next, we need to recreate the model, to indicate that the embeddings will be contained in the batch. We do so by providing a value for the `pretrained_embeddings` in `InputBlockV2`.

In [19]:
embeddings_block = mm.Embeddings(
    loader.output_schema.select_by_tag(Tags.CATEGORICAL).remove_col('movieId'),
)

pretrained_embeddings = mm.PretrainedEmbeddings(
    loader.output_schema.select_by_tag(Tags.EMBEDDING)
)

input_block = mm.InputBlockV2(loader.output_schema, categorical=embeddings_block, pretrained_embeddings=pretrained_embeddings)

model = mm.DCNModel(
    loader.output_schema,
    depth=2,
    input_block=input_block,
    deep_block=mm.MLPBlock([64, 32]),
    prediction_tasks=mm.BinaryOutput(target_column)
)

## Training

Let us train our model.

In [20]:
%%time
opt = tf.keras.optimizers.Adagrad(learning_rate=1e-1)
model.compile(optimizer=opt, run_eagerly=False, metrics=[tf.keras.metrics.AUC()])

model.fit(loader, epochs=5)

In [21]:
history_with_embeddings = model.history.history

The model trains using pretrained embeddings.